## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
# model=keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                            # input_tensor=input_tensor,pooling=None, classes=2)

"""自行填入"""
model = keras.applications.Xception(
    include_top=False,  # 是否導入fullt connected layer
    weights="imagenet",
    input_tensor=input_tensor,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()


Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

## 添加層數

In [2]:
x = model.output

'''可以參考Cifar10實作章節,自行填入'''
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x=Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 136


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)
'''
x_train: 訓練集圖片資料 50000張(32,32,3)圖片
x_test:  測試集圖片資料
y_train: 訓練集圖片分類標籤 (1~10類)
y_test:  測試集圖片分類標籤
'''

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test, mean, std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [5]:
# 載入模型
import os
if os.path.exists("Day020_model/Day020_model.h5"):
    model.load_weights("Day020_model/Day020_model.h5")
else:
    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.fit(x_train,y_train,batch_size=32,epochs=10)

## 預測新圖片

In [6]:
import cv2
from matplotlib import pyplot as plt
import pandas as pd

# 輸入新圖片
# img = cv2.imread("data/Day015_airplane.png")
# img = cv2.imread("data/Day015_car.png")
# img = cv2.imread("data/Day015_bird.png")
# img = cv2.imread("data/Day015_cat.png")
# img = cv2.imread("data/Day015_deer.png")
# img = cv2.imread("data/Day015_dog3.png")
img = cv2.imread("data/Day015_frog.png")
# img = cv2.imread("data/Day015_horse.png")
# img = cv2.imread("data/Day015_ship.png")
# img = cv2.imread("data/Day015_truck.png")
img = cv2.resize(img, (32,32), interpolation = cv2.INTER_BITS) 
plt.imshow(img)

# 前處理 + 預測
input_example=(np.array([img])-mean_train)/(std_train+1e-7) 
result = model.predict(input_example)

# 輸出預測結果
np.set_printoptions(precision=3, suppress=True)
print(result[0] * 100)

# pandas輸出預測結果
d = {'預測結果': result[0] * 100}
idx = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
df = pd.DataFrame(data=d, index = idx)
pd.options.display.float_format = '{:,.2f}%'.format # output options https://stackoverflow.com/a/20937592
df

[ 0.001  0.     0.002  0.001  0.     0.    99.993  0.     0.     0.003]


,預測結果
airplane,0.00%
automobile,0.00%
bird,0.00%
cat,0.00%
deer,0.00%
dog,0.00%
frog,99.99%
horse,0.00%
ship,0.00%
truck,0.00%


In [7]:
## 儲存模型
model.save("Day020_model/Day020_model.h5")

C:\Users\ASUS\Anaconda3\envs\OpenCV_ENV\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


## 心得
1. Keras 儲存與載入訓練好的模型或參數教學 https://blog.gtwang.org/programming/keras-save-and-load-model-tutorial/ <br>
若要將儲存的參數載入至不同的模型中使用（模型不同，但有相同網路層，例如 fine-tuning 或 transfer-learning），可以加上 by_name 參數 <br>
參數就會載入至有相同name的layer中
